In [36]:
from supabase import create_client, Client
import os

model_name = "text-embedding-3-small"
openai_api_key = os.environ['OPENAI_API_KEY']
supabase_url = os.environ['SUPABASE_URL']
supabase_api_key = os.environ['SUPABASE_API_KEY'] 
supabase: Client = create_client(supabase_url, supabase_api_key)

In [9]:
import pandas as pd
page_size = 1000
offset = 0
all_data = []
while True:
    response = supabase.table('multidoc_questions').select("question, answer, gpt_answer, similar_doc_ids, similar_doc_ids_bm25, similar_doc_ids_tfidf").range(offset, offset + page_size - 1).execute()
    if not response.data:
        break
    all_data.extend(response.data)
    offset += page_size
gpt_answer_result = pd.DataFrame(all_data)


In [10]:
page_size = 1000
offset = 0
all_data = []
while True:
    response = supabase.table('multidoc_sources').select("id, docid, title, content").range(offset, offset + page_size - 1).execute()
    if not response.data:
        break
    all_data.extend(response.data)
    offset += page_size
gpt_source_result = pd.DataFrame(all_data)

In [11]:
# 將document的編號轉成黏在一起的文字字串，需要用gpt_source_result對應
# Similar_doc_ids to document_ids
import time
start = time.time()
gpt_answer_result["contexts"] = None
for index, row in gpt_answer_result.iterrows():
    similar_doc_ids = row['similar_doc_ids']
    if similar_doc_ids:
        final_content = ""
        for chunk_id in similar_doc_ids:
            original_chunk = gpt_source_result[gpt_source_result['id'] == chunk_id]["content"].values[0]
            final_content += original_chunk
        gpt_answer_result.at[index, 'contexts'] = final_content
end = time.time()
print(f"Time taken : {end - start} seconds")


Time taken : 0.4150381088256836 seconds


In [33]:
df = gpt_answer_result[['question', 'gpt_answer', 'answer', 'contexts']]
df = df.sample(n=100, random_state=42)  # random_state 可以確保結果可重現
df

,question,gpt_answer,answer,contexts
644,兒童食用零食的主要原因是什麼？,兒童食用零食的主要原因是因為零食好吃，並且吃了容易上癮。此外，兒童伙伴之間的攀比心態也促使了...,根據原文“據有關部門的調查數據顯示，在我國，兒童食用零食的主要原因很簡單，因爲好喫，並且喫了...,兒童零食 首次對兒童零食安全性提出要求：兒童零食的食品添加劑指標向嬰幼兒輔食標準看齊，不允許...
629,鄉賢是什麼？,鄉賢是指品德和才學受到鄉人推崇敬重的人。這一詞語始於東漢，是國家對有作爲的官員或有崇高威望、...,根據原文“鄉賢，漢語詞語，釋義爲品德、才學爲鄉人推崇敬重的人。”可知，鄉賢是指品德、才學受到...,鄉賢 鄉賢，漢語詞語，釋義爲品德、才學爲鄉人推崇敬重的人。鄉賢釋義和出處 品德、才學爲鄉人推...
70,三腳兩步這個成語出自哪裏？,三腳兩步這個成語出自清·李寶嘉《官場現形記》。,根據原文“成語出處 清·李寶嘉《官場現形記》第十回：‘太太三腳兩步，走進房間。’”可知，三腳...,三腳兩步 三腳兩步，漢語成語，拼音是sān jiǎo liǎng bù，意思是走得快，走不多...
962,什麼是作曲軟件中的軟音源？,作曲軟件中的軟音源，也稱為“虛擬樂器”，是指用來產生各種音色的軟件，是一種軟件化的樂器。軟音...,根據原文“軟音源 用來產生各種音色的軟件。又稱‘虛擬樂器’，是軟件合成器的另一種形式，就是軟...,作曲軟件 有些軟件本身是DAW但也可以當軟音源來使用，但一般不作爲插件使用。有些軟件既可以獨...
787,一片無痕文胸的特點是什麼？,一片無痕文胸的特點包括：\n\n1. 超細纖維布料製成，柔滑輕薄。\n2. 采用粘合技術，罩...,根據原文“一片無痕文胸是用超細纖維布料製成的，摸上去十分柔滑輕薄，直接一次成型爲表面光滑流暢...,一片無痕文胸 一片無痕文胸生產工藝 一片式無痕文胸是用超細纖維布料製成的，摸上去十分柔滑輕薄...
...,...,...,...,...
352,假面愛人遊戲的作者是誰？,假面愛人遊戲的作者是米斯特（@mister2）。,根據原文“《假面愛人》是一款校園文字遊戲，遊戲作者是米斯特（@mister2）。”可知，假面...,假面愛人 《假面愛人》是一款校園文字遊戲，遊戲作者是米斯特（@mister2）。假面愛人遊戲...
49,三養是指什麼？,三養是指養福、養氣、養財。,根據原文“三養，讀音sān yǎng，漢語詞語，謂養福、養氣、養財。條目：三養（三養） 解釋...,三養 三養，讀音sān yǎng，漢語詞語，謂養福、養氣、養財。條目：三養（三養） 解釋： ...
141,《中國民俗老黃曆》的作者是誰？,《中國民俗老黃曆》的作者是任丙未。,根據原文“《中國民俗老黃曆》由氣象出版社出版，作者任丙未”可知，《中國民俗老黃曆》的作者是任丙未。,中國民俗老黃曆 《中國民俗老黃曆》由氣象出版社出版，作者任丙未。老黃曆是中國傳統民俗文化的特...
198,八里關鎮的人口數量和行政區劃情況是怎樣的？,八里關鎮的人口數量為3429人（2017年數據），轄有8個行政村和46個村民小組。行政區劃包...,根據原文“八里關鎮地處洋縣北部，屬中國氣候南北分界線－秦嶺腹地，平均海拔700米，距縣城47...,八里關鎮 八里關鎮地處洋縣北部，屬中國氣候南北分界線－秦嶺腹地，平均海拔700米，距縣城47...


In [34]:
from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(df)

# 2. 創建 DatasetDict
dataset_dict = DatasetDict({
    'baseline': dataset
})
dataset_dict['baseline'] = dataset_dict['baseline'].rename_column('answer', 'reference').rename_column('gpt_answer', 'answer')
def preprocess_dataset(dataset: Dataset) -> Dataset:
    """預處理數據集，確保格式正確"""
    
    def process_row(row):
        # 處理 contexts，確保它是列表格式
        if isinstance(row['contexts'], str):
            # 如果是字符串，將其轉換為單元素列表
            contexts = [row['contexts']]
        elif isinstance(row['contexts'], list):
            contexts = row['contexts']
        else:
            contexts = []
            
        # 創建新的行數據
        return {
            'question': row['question'],
            'answer': row['answer'],
            'reference': row['reference'],
            'contexts': contexts,
            # 如果需要保留 index，也可以保留
            '__index_level_0__': row.get('__index_level_0__', 0)
        }
    
    # 應用處理函數到每一行
    processed_data = [process_row(row) for row in dataset]
    
    # 創建新的 Dataset
    return Dataset.from_pandas(pd.DataFrame(processed_data))
dataset_dict = preprocess_dataset(dataset_dict['baseline'])


In [27]:
dataset_dict

Dataset({
    features: ['question', 'answer', 'reference', 'contexts', '__index_level_0__'],
    num_rows: 100
})

In [21]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)

In [40]:
from ragas import evaluate
import time
from datetime import datetime
import pandas as pd
import numpy as np

# 創建空的DataFrame來存儲結果
results_df = pd.DataFrame(columns=[
    'batch_number',
    'start_time',
    'end_time',
    'execution_time',
    'context_precision',
    'faithfulness',
    'answer_relevancy',
    'context_recall'
])

# 計算總批次數（每批10個樣本）
total_samples = len(dataset_dict)
batch_size = 10
num_batches = 10

for batch in range(num_batches):
    # 記錄開始時間
    start_time = time.time()
    start_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"\n批次 {batch + 1}/{num_batches}")
    print(f"評估開始時間: {start_datetime}")
    
    # 選擇當前批次的數據
    start_idx = batch * batch_size
    end_idx = start_idx + batch_size
    batch_data = dataset_dict.select(range(start_idx, end_idx))
    
    try:
        # 執行評估
        result = evaluate(
            batch_data,
            metrics=[
                context_precision,
                faithfulness,
                answer_relevancy,
                context_recall
            ],
        )
        
        # 記錄結束時間
        end_time = time.time()
        end_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        execution_time = end_time - start_time
        
        # 將結果添加到DataFrame
        results_df.loc[batch] = {
            'batch_number': batch + 1,
            'start_time': start_datetime,
            'end_time': end_datetime,
            'execution_time': execution_time,
            'context_precision': result['context_precision'],
            'faithfulness': result['faithfulness'],
            'answer_relevancy': result['answer_relevancy'],
            'context_recall': result['context_recall']
        }
        
        print(f"評估結束時間: {end_datetime}")
        print(f"執行時間: {execution_time:.2f} 秒")
        print("\n當前批次結果:")
        print(result)
        
        # 保存中間結果到CSV
        results_df.to_csv('ragas_evaluation_results.csv', index=False)
        
    except Exception as e:
        print(f"批次 {batch + 1} 發生錯誤: {str(e)}")
        continue




批次 1/10
評估開始時間: 2024-12-22 15:16:14


Evaluating: 100%|██████████| 40/40 [00:27<00:00,  1.47it/s]


評估結束時間: 2024-12-22 15:16:43
執行時間: 28.74 秒

當前批次結果:
{'context_precision': 0.9000, 'faithfulness': 0.8218, 'answer_relevancy': 0.6782, 'context_recall': 1.0000}

批次 2/10
評估開始時間: 2024-12-22 15:16:43


Evaluating: 100%|██████████| 40/40 [00:38<00:00,  1.04it/s]


評估結束時間: 2024-12-22 15:17:22
執行時間: 39.01 秒

當前批次結果:
{'context_precision': 1.0000, 'faithfulness': 0.9000, 'answer_relevancy': 0.8938, 'context_recall': 0.9750}

批次 3/10
評估開始時間: 2024-12-22 15:17:22


...

Evaluating: 100%|██████████| 40/40 [00:30<00:00,  1.30it/s]


評估結束時間: 2024-12-22 15:21:38
執行時間: 32.18 秒

當前批次結果:
{'context_precision': 0.6000, 'faithfulness': 0.7524, 'answer_relevancy': 0.5200, 'context_recall': 0.6750}

批次 10/10
評估開始時間: 2024-12-22 15:21:38


Evaluating: 100%|██████████| 40/40 [01:32<00:00,  2.31s/it]


評估結束時間: 2024-12-22 15:23:11
執行時間: 92.97 秒

當前批次結果:
{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.8744, 'context_recall': 0.9667}

所有批次的平均結果:


TypeError: Could not convert [list([0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.0, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.0, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.0, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.0, 0.9999999999, 0.9999

In [45]:
#跑了202次?因為有一次失敗了
df_ragas = result.to_pandas()
print(df_ragas)

              user_input                                 retrieved_contexts  \
0    上畛子革命舊址的保護範圍包括哪些內容？  [上畛子革命舊址 上畛子革命舊址，時代：1931—1937年，地址：延安市黃陵縣雙龍鎮林灣村...   
1            餘新顏擔任過哪些職務？  [餘新顏 餘新顏，女，1976年1月出生，四川南江人，1996年6月加入中國共產黨，1997...   
2              什麼是全液壓鑽機？  [全液壓鑽機 全液壓鑽機（all-hydraulic operated drill）也稱全油...   
3            絲帶切帶機有哪些特點？  [絲帶切帶機 絲帶切帶機是利用超聲波熔接技術與傳統剪切相結合，在超聲波發生器工作時，將超聲波...   
4     《也許吧》是哪裏連載的一部都市小說？  [也許吧 《也許吧》是連載於新浪讀書網的一部都市小說，作者是小四朵。小說類型 都市小說 內容...   
5           假面愛人遊戲的作者是誰？  [假面愛人 《假面愛人》是一款校園文字遊戲，遊戲作者是米斯特（@mister2）。假面愛人遊...   
6                三養是指什麼？  [三養 三養，讀音sān yǎng，漢語詞語，謂養福、養氣、養財。條目：三養（三養） 解釋：...   
7        《中國民俗老黃曆》的作者是誰？  [中國民俗老黃曆 《中國民俗老黃曆》由氣象出版社出版，作者任丙未。老黃曆是中國傳統民俗文化的...   
8  八里關鎮的人口數量和行政區劃情況是怎樣的？  [八里關鎮 八里關鎮地處洋縣北部，屬中國氣候南北分界線－秦嶺腹地，平均海拔700米，距縣城4...   
9         在臺灣，是否設立了兒童法庭？  [兒童法庭 兒童法庭，作爲一般法庭的對照，是用以區別審理少年犯罪者與成人犯罪者的特殊法庭。 ...   

                                            response  \
0  上畛子革命舊址的保護範圍包括A區和B區：\n\n- A區：\n  1. 高窯子革命舊址：舊址...   
1  餘新顏擔任過的職務包括：\n\

In [51]:
import pandas as pd
import ast
import numpy as np

# 讀取CSV文件
results_df = pd.read_csv('ragas_evaluation_results.csv')

# 初始化結果變數
total_execution_time = 0
all_context_precision = []
all_faithfulness = []
all_answer_relevancy = []
all_context_recall = []

# 遍歷每一行數據
for index, row in results_df.iterrows():
    # 累加執行時間
    total_execution_time += row['execution_time']
    
    # 將字符串形式的列表轉換為實際的列表
    context_precision = ast.literal_eval(row['context_precision'])
    faithfulness = ast.literal_eval(row['faithfulness'])
    answer_relevancy = ast.literal_eval(row['answer_relevancy'])
    context_recall = ast.literal_eval(row['context_recall'])
    
    # 將每個批次的結果添加到對應的列表中
    all_context_precision.extend(context_precision)
    all_faithfulness.extend(faithfulness)
    all_answer_relevancy.extend(answer_relevancy)
    all_context_recall.extend(context_recall)

# 計算平均值
avg_context_precision = np.mean(all_context_precision)
avg_faithfulness = np.mean(all_faithfulness)
avg_answer_relevancy = np.mean(all_answer_relevancy)
avg_context_recall = np.mean(all_context_recall)

# 輸出結果
print("\n所有批次的評估結果:")
print(f"總執行時間: {total_execution_time:.2f} 秒")
print(f"平均 Context Precision: {avg_context_precision:.4f}")
print(f"平均 Faithfulness: {avg_faithfulness:.4f}")
print(f"平均 Answer Relevancy: {avg_answer_relevancy:.4f}")
print(f"平均 Context Recall: {avg_context_recall:.4f}")

# 將最終結果保存到新的DataFrame
final_results = pd.DataFrame({
    'metric': ['total_execution_time', 'avg_context_precision', 'avg_faithfulness', 'avg_answer_relevancy', 'avg_context_recall'],
    'value': [total_execution_time, avg_context_precision, avg_faithfulness, avg_answer_relevancy, avg_context_recall]
})

# 保存最終結果
final_results.to_csv('ragas_evaluation_summary.csv', index=False)


所有批次的平均結果:
[0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999, 0.0, 0.9999999999, 0.9999999999, 0.9999999999, 0.9999999999]
